#### Steps

- Import the CSV file into a pandas df
- Clean the table name and remove all extra sysmbols, spaces, capital letter
- Clean the colum headers and remove all extra symbols, spaces and capital letter
- Write the create tbale SQL statement
- Import the data into the db

In [2]:
# importing libraries
# !pip install psycopg2

import os
import numpy as np
import pandas as pd
import psycopg2

# Read the csv file
df = pd.read_csv('WDIData.csv')

# Remove the last column that contains all the unnamed columns
# df = df.iloc[:, :-1]

df.columns = ['yr' + col if col not in df.columns[:4] and not col.startswith('yr') else col for col in df.columns]
df.columns = [x.lower().replace(" ", "_") for x in df.columns]
df = df.filter(regex='^(?!yrunnamed)')
df.fillna(0, inplace=True)

replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int32' : 'int',
    'datetime64' : 'timestamp'
}

col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))

# conn_string = 'host="localhost", port="5432", dbname="WDI", user="postgres", password="admin"' 
conn = psycopg2.connect(host="localhost", port="5432", dbname="WDI", user="postgres", password="admin")
cursor = conn.cursor()
print('Opened databse successfully')

# drop table with same name
cursor.execute("drop table if exists WDIData;")

cursor.execute("create table WDIData(%s)" % (col_str))

# save df to csv
df.to_csv('wdidata.csv', header=df.columns, index=False, encoding='utf-8')

#open the csv file

my_file = open('wdidata.csv')
print('file opened in memory')

SQL_STATEMENT = """
COPY wdidata FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print('file copied to db')

cursor.execute("grant select on WDIData to public")
conn.commit()
cursor.close()
print('table WDIData import to db completed')

Opened databse successfully
file opened in memory
file copied to db
table WDIData import to db completed
